In [29]:
import pandas as pd
import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lesty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
courses = pd.read_csv('Data/courses_dataset.csv')
courses = courses.drop(['Unnamed: 0'], axis=1)

In [23]:
courses

,course_id,url,title,translated_title,is_paid,instructors,translated_instructors,image_480x270,published_title,headline,...,created_at,instructional_level,published_at,objectives_summary,translated_objectives_summary,is_recently_published,last_update_date,content_info,category,char_count
0,655608,/course/productivity-and-time-management/,Productivity and Time Management for the Overw...,Productivity and Time Management for the Overw...,True,['Josh Paulsen'],['Josh Paulsen'],https://img-b.udemycdn.com/course/480x270/6556...,productivity-and-time-management,A complete productivity and time management co...,...,2015-10-30T15:09:33Z,All Levels,2015-11-10T22:18:22Z,['Take control of their mindset and energy lev...,['Take control of their mindset and energy lev...,False,2017-06-14,2 total hours,personal_development,254.0
1,591930,/course/become-a-superlearner-2-speed-reading-...,Become a SuperLearner® 2: Learn Speed Reading ...,Become a SuperLearner® 2: Learn Speed Reading ...,True,"['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...","['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...",https://img-b.udemycdn.com/course/480x270/5919...,become-a-superlearner-2-speed-reading-memory-a...,The original course to learn faster & more eas...,...,2015-08-24T15:22:20Z,All Levels,2015-12-03T18:33:51Z,['Learn to read roughly 3 times the speed of a...,['Learn to read roughly 3 times the speed of a...,False,2021-10-08,5.5 total hours,personal_development,414.0
2,1080376,/course/life-coaching-online-certification-cou...,Life Coaching Certificate Course (Beginner to ...,Life Coaching Certificate Course (Beginner to ...,True,"['Kain Ramsay', 'The Academy of Modern Applied...","['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1080...,life-coaching-online-certification-course-life...,A comprehensive online training that delivers ...,...,2017-01-17T22:38:10Z,All Levels,2017-03-11T03:58:33Z,['A time-proven life coaching methodology that...,['A time-proven life coaching methodology that...,False,2022-01-24,33 total hours,personal_development,255.0
3,2108272,/course/do-more-stress-less/,"Time Management Mastery: Do More, Stress Less","Time Management Mastery: Do More, Stress Less",True,['Alexis Haselberger'],['Alexis Haselberger'],https://img-b.udemycdn.com/course/480x270/2108...,do-more-stress-less,"Take control of your time, your to-do list, an...",...,2018-12-26T17:41:09Z,All Levels,2019-04-11T17:04:22Z,['Discover and apply time-management and produ...,['Discover and apply time-management and produ...,False,2021-10-21,2 total hours,personal_development,279.0
4,1336872,/course/nlp-practitioner-neuro-linguistic-prog...,Neuro-Linguistic Programming (NLP) Practitione...,Neuro-Linguistic Programming (NLP) Practitione...,True,"['Kain Ramsay', 'The Academy of Modern Applied...","['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1336...,nlp-practitioner-neuro-linguistic-programming-...,Gain an in-depth understanding of Neuro-Lingui...,...,2017-08-31T16:14:17Z,All Levels,2017-10-09T16:23:41Z,['The conscious use of language – Milton & Met...,['The conscious use of language – Milton & Met...,False,2021-12-10,44 total hours,personal_development,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111042,4330136,/course/gimp-dinosaur/,初心者におすすめ無料の画像編集ソフトGIMPで恐竜を粘土細工のように簡単に作れる方法を学べます！,You can learn how to make dinosaurs easily lik...,False,['Yuji Kuroki'],['YUJI KUROKI'],https://img-c.udemycdn.com/course/480x270/4330...,gimp-dinosaur,画像編集ソフト使ったことのない初心者の方や絵が描けない方でも大丈夫です！作品を作りながらGI...,...,2021-10-02T06:21:33Z,Beginner Level,2021-10-08T12:59:38Z,"['画像編集ソフトGIMPの使い方を作品を作りながら覚えられる！', '初心者でも絵が描けな...",['You can learn how to use the image editing s...,False,2022-02-01,1.5 total hours,photography,NaN
111043,1465960,/course/gopro-prise-en-main-materiel-et-logici...,"GOPRO, Prise en main MATERIEL et LOGICIELS","Goopro, Material handling and software",False

In [3]:
# All Translated
# Translated Title
# Translated Instructors
# Translated Headlines
# Translated Objectives Summary

In [24]:
# Check NaN Values
def check_nan(df):
    nan_columns = df.columns[df.isna().any()].tolist()
    if nan_columns:
        print("NaN values found in the following columns:")
        for col in nan_columns:
            print(f"{col}: {df[col].isna().sum()} NaN values")
    else:
        print("No NaN values found in any columns")

In [25]:
check_nan(courses)

NaN values found in the following columns:
published_title: 1 NaN values
headline: 13 NaN values
translated_headline: 13 NaN values
instructional_level: 17 NaN values
last_update_date: 144 NaN values
char_count: 101793 NaN values


In [98]:
final_df = courses.copy()

In [99]:
# Since we have 13 and/or 17 data with NaN values, so we are going to drop the data
final_df = final_df.dropna(subset=['headline'])
final_df = final_df.dropna(subset=['instructional_level'])

check_nan(final_df)

NaN values found in the following columns:
published_title: 1 NaN values
last_update_date: 144 NaN values
char_count: 101766 NaN values


## Function For EDA

In [66]:
def displaylength(data, kolom):
  # Convert course description to string type
  data[kolom] = data[kolom].astype(str)

  # Calculate length of course description
  data["Length_Characters"] = data[kolom].apply(lambda text: len(text))

  # Hitung jumlah kata
  data["Length_Words"] = data[kolom].apply(lambda text: len(text.split()))

  # Hitung panjang rata-rata
  print(np.mean(data["Length_Characters"]))
  print(np.mean(data["Length_Words"]))
  print(np.max(data["Length_Characters"]))
  print(np.min(data["Length_Characters"]))
  print(np.max(data["Length_Words"]))
  print(np.min(data["Length_Words"]))

def wordfrequencyanalysis(data, kolom):
  words = data[kolom].str.split(expand=True).stack()
  word_counts = words.value_counts()
  top_10_words = word_counts.head(10)
  print("Top 10 words:\n")
  return top_10_words

def phrasefrequencyanalysis(data, kolom):
  vectorizer = CountVectorizer(ngram_range=(2, 2))
  phrases = vectorizer.fit_transform(data[kolom])
  phrase_counts = pd.DataFrame(phrases.sum(axis=0), columns=vectorizer.get_feature_names_out())
  top_10_phrases = phrase_counts.T.sort_values(by=0, ascending=False).head(10)

  # Rename the columns to 'phrases' and 'frequency'
  top_10_phrases = top_10_phrases.rename(columns={0: 'frequency'})
  top_10_phrases.index.name = 'phrases'

  print("\nTop 10 phrases:\n")
  return top_10_phrases

## Function For Preprocessing

In [65]:
def remove_punctuation(text):
    # Create a regular expression pattern that matches any character that is not a word character or a whitespace character.
    punct_pattern = re.compile(r'[^\w\s]')

    # Substitute the matched characters with a single space.
    return punct_pattern.sub(' ', text)

def remove_stopwords(text):
    # Create a set of English stopwords.
    stop_words = set(stopwords.words('english'))

    # Remove stopwords from the text.
    filtered_text = [word for word in text.split() if word not in stop_words]

    # Join the filtered words back into a single string.
    return ' '.join(filtered_text)

def remove_extra_whitespaces(text):
    # Create a regular expression pattern that matches one or more whitespace characters.
    whitespace_pattern = re.compile(r'\s+')

    # Substitute the matched whitespace characters with a single space.
    return whitespace_pattern.sub(' ', text)

def remove_numbers(text):
    # Create a regular expression pattern that matches any number
    number_pattern = re.compile(r'\d+')

    # Substitute the matched numbers with an empty string
    return number_pattern.sub(' ', text)

def stem_text_porter(text):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text.split()]
    return ' '.join(stemmed_words)

In [91]:
def preprocessing_with_stemming(df, column):
    # Case Folding
    df['preprocessed_description_stemmed'] = df[column].str.lower()

    # Remove Punctuations
    df['preprocessed_description_stemmed'] = df['preprocessed_description_stemmed'].apply(remove_punctuation)

    # Remove Numbers
    df['preprocessed_description_stemmed'] = df['preprocessed_description_stemmed'].apply(remove_numbers)

    # Remove Stop Words
    df['preprocessed_description_stemmed'] = df['preprocessed_description_stemmed'].apply(remove_stopwords)

    # Replace Multiple Whitespaces with a Single Space
    df['preprocessed_description_stemmed'] = df['preprocessed_description_stemmed'].apply(remove_extra_whitespaces)

    # Stemming
    df['preprocessed_description_stemmed'] = df['preprocessed_description_stemmed'].apply(stem_text_porter)

    return df

def preprocessing_without_stemming(df, column):
    # Case Folding
    df['preprocessed_description_unstemmed'] = df[column].str.lower()

    # Remove Punctuations
    df['preprocessed_description_unstemmed'] = df['preprocessed_description_unstemmed'].apply(remove_punctuation)

    # Remove Numbers
    df['preprocessed_description_unstemmed'] = df['preprocessed_description_unstemmed'].apply(remove_numbers)

    # Remove Stop Words
    df['preprocessed_description_unstemmed'] = df['preprocessed_description_unstemmed'].apply(remove_stopwords)

    # Replace Multiple Whitespaces with a Single Space
    df['preprocessed_description_unstemmed'] = df['preprocessed_description_unstemmed'].apply(remove_extra_whitespaces)

    return df

In [82]:
def feature_preprocess(series_text, case_folding=False, normalization=False, tokenize=False, rm_stopwords = False, stemming = False):
    series_text = pd.Series(series_text)

    if case_folding == True:
        series_text = series_text.str.lower()

    if normalization == True:
        # It will replaces one or more whitespace characters with a single space and 
        # replaces any character that is not in the range a-z with a space in each element 
        series_text = series_text.str.replace(r"\s+"," ")
        series_text = series_text.str.replace("[^a-z]"," ")

    if rm_stopwords == True:
        stopwords = nltk.corpus.stopwords.words('english')
        for sw in stopwords:
            regex_stopword = r"\b" + sw + r"\b"
            series_text = series_text.str.replace(regex_stopword,'')

    if stemming == True:
        # It will reduce words to their root or base form by removing common morphological and inflectional endings.
        stemmer = PorterStemmer()
        def stemming(text, stemmer):
            return stemmer.stem(text)

        series_text = series_text.map(lambda text: stemming(text, stemmer))

    if tokenize == True:
        series_text = series_text.apply(word_tokenize)
    return series_text.tolist()

## Preprocessing

In [100]:
final_df['description'] = final_df['translated_title'] + " " + final_df['translated_headline'] + " " + final_df['translated_objectives_summary'] + " " + final_df['instructional_level']

In [101]:
preprocessing_with_stemming(final_df, 'description')

,course_id,url,title,translated_title,is_paid,instructors,translated_instructors,image_480x270,published_title,headline,...,published_at,objectives_summary,translated_objectives_summary,is_recently_published,last_update_date,content_info,category,char_count,description,preprocessed_description_stemmed
0,655608,/course/productivity-and-time-management/,Productivity and Time Management for the Overw...,Productivity and Time Management for the Overw...,True,['Josh Paulsen'],['Josh Paulsen'],https://img-b.udemycdn.com/course/480x270/6556...,productivity-and-time-management,A complete productivity and time management co...,...,2015-11-10T22:18:22Z,['Take control of their mindset and energy lev...,['Take control of their mindset and energy lev...,False,2017-06-14,2 total hours,personal_development,254.0,Productivity and Time Management for the Overw...,product time manag overwhelm complet product t...
1,591930,/course/become-a-superlearner-2-speed-reading-...,Become a SuperLearner® 2: Learn Speed Reading ...,Become a SuperLearner® 2: Learn Speed Reading ...,True,"['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...","['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...",https://img-b.udemycdn.com/course/480x270/5919...,become-a-superlearner-2-speed-reading-memory-a...,The original course to learn faster & more eas...,...,2015-12-03T18:33:51Z,['Learn to read roughly 3 times the speed of a...,['Learn to read roughly 3 times the speed of a...,False,2021-10-08,5.5 total hours,personal_development,414.0,Become a SuperLearner® 2: Learn Speed Reading ...,becom superlearn learn speed read boost memori...
2,1080376,/course/life-coaching-online-certification-cou...,Life Coaching Certificate Course (Beginner to ...,Life Coaching Certificate Course (Beginner to ...,True,"['Kain Ramsay', 'The Academy of Modern Applied...","['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1080...,life-coaching-online-certification-course-life...,A comprehensive online training that delivers ...,...,2017-03-11T03:58:33Z,['A time-proven life coaching methodology that...,['A time-proven life coaching methodology that...,False,2022-01-24,33 total hours,personal_development,255.0,Life Coaching Certificate Course (Beginner to ...,life coach certif cours beginn intermedi compr...
3,2108272,/course/do-more-stress-less/,"Time Management Mastery: Do More, Stress Less","Time Management Mastery: Do More, Stress Less",True,['Alexis Haselberger'],['Alexis Haselberger'],https://img-b.udemycdn.com/course/480x270/2108...,do-more-stress-less,"Take control of your time, your to-do list, an...",...,2019-04-11T17:04:22Z,['Discover and apply time-management and produ...,['Discover and apply time-management and produ...,False,2021-10-21,2 total hours,personal_development,279.0,"Time Management Mastery: Do More, Stress Less ...",time manag masteri stress less take control ti...
4,1336872,/course/nlp-practitioner-neuro-linguistic-prog...,Neuro-Linguistic Programming (NLP) Practitione...,Neuro-Linguistic Programming (NLP) Practitione...,True,"['Kain Ramsay', 'The Academy of Modern Applied...","['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1336...,nlp-practitioner-neuro-linguistic-programming-...,Gain an in-depth understanding of Neuro-Lingui...,...,2017-10-09T16:23:41Z,['The conscious use of language – Milton & Met...,['The conscious use of language – Milton & Met...,False,2021-12-10,44 total hours,personal_development,202.0,Neuro-Linguistic Programming (NLP) Practitione...,neuro linguist program nlp practition certif g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111042,4330136,/course/gimp-dinosaur/,初心者におすすめ無料の画像編集ソフトGIMPで恐竜を粘土細工のように簡単に作れる方法を学べます！,You can learn how to make dinosaurs easily lik...,False,['Yuji Kuroki'],['YUJI KUROKI'],https://img-c.udemycdn.com/course/480x270/4330...,gimp-dinosaur,画像編集ソフト使ったことのない初心者の方や絵が描けない方でも大丈夫です！作品を作りながらGI...,...,2021-10-

In [102]:
preprocessing_without_stemming(final_df, 'description')

,course_id,url,title,translated_title,is_paid,instructors,translated_instructors,image_480x270,published_title,headline,...,objectives_summary,translated_objectives_summary,is_recently_published,last_update_date,content_info,category,char_count,description,preprocessed_description_stemmed,preprocessed_description_unstemmed
0,655608,/course/productivity-and-time-management/,Productivity and Time Management for the Overw...,Productivity and Time Management for the Overw...,True,['Josh Paulsen'],['Josh Paulsen'],https://img-b.udemycdn.com/course/480x270/6556...,productivity-and-time-management,A complete productivity and time management co...,...,['Take control of their mindset and energy lev...,['Take control of their mindset and energy lev...,False,2017-06-14,2 total hours,personal_development,254.0,Productivity and Time Management for the Overw...,product time manag overwhelm complet product t...,productivity time management overwhelmed compl...
1,591930,/course/become-a-superlearner-2-speed-reading-...,Become a SuperLearner® 2: Learn Speed Reading ...,Become a SuperLearner® 2: Learn Speed Reading ...,True,"['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...","['Jonathan Levi', 'Lev Goldentouch', 'Anna Gol...",https://img-b.udemycdn.com/course/480x270/5919...,become-a-superlearner-2-speed-reading-memory-a...,The original course to learn faster & more eas...,...,['Learn to read roughly 3 times the speed of a...,['Learn to read roughly 3 times the speed of a...,False,2021-10-08,5.5 total hours,personal_development,414.0,Become a SuperLearner® 2: Learn Speed Reading ...,becom superlearn learn speed read boost memori...,become superlearner learn speed reading boost ...
2,1080376,/course/life-coaching-online-certification-cou...,Life Coaching Certificate Course (Beginner to ...,Life Coaching Certificate Course (Beginner to ...,True,"['Kain Ramsay', 'The Academy of Modern Applied...","['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1080...,life-coaching-online-certification-course-life...,A comprehensive online training that delivers ...,...,['A time-proven life coaching methodology that...,['A time-proven life coaching methodology that...,False,2022-01-24,33 total hours,personal_development,255.0,Life Coaching Certificate Course (Beginner to ...,life coach certif cours beginn intermedi compr...,life coaching certificate course beginner inte...
3,2108272,/course/do-more-stress-less/,"Time Management Mastery: Do More, Stress Less","Time Management Mastery: Do More, Stress Less",True,['Alexis Haselberger'],['Alexis Haselberger'],https://img-b.udemycdn.com/course/480x270/2108...,do-more-stress-less,"Take control of your time, your to-do list, an...",...,['Discover and apply time-management and produ...,['Discover and apply time-management and produ...,False,2021-10-21,2 total hours,personal_development,279.0,"Time Management Mastery: Do More, Stress Less ...",time manag masteri stress less take control ti...,time management mastery stress less take contr...
4,1336872,/course/nlp-practitioner-neuro-linguistic-prog...,Neuro-Linguistic Programming (NLP) Practitione...,Neuro-Linguistic Programming (NLP) Practitione...,True,"['Kain Ramsay', 'The Academy of Modern Applied...","['Kain Ramsay', 'The Academy of Modern Applied...",https://img-b.udemycdn.com/course/480x270/1336...,nlp-practitioner-neuro-linguistic-programming-...,Gain an in-depth understanding of Neuro-Lingui...,...,['The conscious use of language – Milton & Met...,['The conscious use of language – Milton & Met...,False,2021-12-10,44 total hours,personal_development,202.0,Neuro-Linguistic Programming (NLP) Practitione...,neuro linguist program nlp practition certif g...,neuro linguistic programming nlp practitioner ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111042,4330136,/course/gimp-dinosaur/,初心者におすすめ無料の画像編集ソフトGIMPで恐竜を粘土細工のように簡単に作れる方法を学べます！,You can learn how to make dinosaurs easily lik.

## Export File

In [103]:
final_df.to_csv('Data/preprocessed_courses.csv')

In [104]:
print("NLTK version:", nltk.__version__)

NLTK version: 3.8.1


In [1]:
import torch

In [2]:
torch.cuda.is_available()

False